In [1]:
import pandas as pd # type: ignore
import firebase_admin # type: ignore
from firebase_admin import credentials, firestore # type: ignore

cred = credentials.Certificate('../../Documents/GoogleCerts/astroworld-trevor-firebase-adminsdk-l05ps-820638c3ff.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

def name_star(row):
    name = row['proper']
    hd = row['hd']
    if pd.isna(name):
        if pd.isna(hd):
            return None
        else:
            return f"HD{int(hd)}"
    else:
        return name

def convert_dist(parsecs):
    au = parsecs * 206264.80624
    return au

table = pd.read_csv('./hygdata_v41.csv')
table = table.sort_values('mag')
table = table[table['proper'] != 'Sol']
table = table.head(500)
table['proper'] = table.apply(name_star, axis=1)
table['dist'] = table['dist'].apply(convert_dist)
stars_table = table[['id','proper','ra','dec','mag','spect','dist','con','lum']].rename(columns={
    'id': 'id',
    'proper': 'name',
    'ra': 'rightAscension',
    'dec': 'declination',
    'mag': 'magnitude',
    'spect': 'spectralType',
    'dist':'distance',
    'con':'constellation',
    'lum':'luminosity'
})

stars_dict = stars_table.to_dict(orient='records')
stars_ref = db.collection('stars')
for star in stars_dict:
    star_ref = stars_ref.document(str(int(star['id'])))
    star_ref.set(star)

print(f"{len(stars_dict)} stars uploaded successfully")

500 stars uploaded successfully
